In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
import numpy as np
import itertools
import pandas as pd

from konlpy.tag import Okt

In [6]:
def listToString(s):  
    str1 = ""  
    for ele in s:  
        str1 += " " + ele.strip()
    return str1

def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]


def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [2]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens') # OK / 10분?

In [4]:
# Data Load
df = pd.read_excel('./20230327_review_data5_temp.xlsx', 0)

In [7]:
# Text Cleansing
try:
    doc = listToString(df.REVIEW.sample(frac = 1))
except:
    doc = listToString(df.리뷰상세내용.sample(frac = 1))
    pass

In [10]:
okt = Okt()
tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if (word[1] == 'Noun') | (word[1] == 'Verb') | (word[1] == 'Adjective')])

# 2음절 ~ 3음절
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('bi/trigram 개수 :',len(candidates))
print('bi/trigram 10개만 출력 :',candidates[0:9])

trigram 개수 : 15549
trigram 10개만 출력 : ['가격 같아서' '가격 같아서 만족합니다' '가격 구매' '가격 구매 해서' '가격 구매 했어요' '가격 구입'
 '가격 구입 있어서' '가격 구입 했어요' '가격 내려가']


In [11]:
# 문서간 유사도
doc_embedding = model.encode([doc])

# 단어간 유사도 : 30분 초과시 pass
candidate_embeddings = model.encode(candidates)

In [12]:
top_n = 15
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords3 = mmr(doc_embedding, candidate_embeddings, candidates, top_n=15, diversity=0.8)
keywords2 =  max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=15, nr_candidates=20)

df_k = pd.DataFrame()
df_k['BERT'] =  keywords
df_k['BERT_MMS'] =  keywords2
df_k['BERT_MMR'] =  keywords3

In [13]:
df_k

,BERT,BERT_MMS,BERT_MMR
0,깔끔한 포장,포장 넉넉한 유통,좋아합니다 맛있고 포장
1,맛있어요 쿠키 먹어,꼼꼼하고 상품 터짐,우유 유통 기한
2,두유 맛있어요 먹겠습니다,건강한 쿠키,감자 불고기
3,넉넉한 깨끗한 물품,깔끔한 포장,마음 매일유업
4,진쪼 좋아합니다 맛있고,맛있어요 쿠키 먹어,상자 자녀
5,제품 건강한 단맛,두유 맛있어요 먹겠습니다,다른 유통업체
6,맛있어요 간편하게 먹을수있어요,넉넉한 깨끗한 물품,마트 인터넷
7,빠르고 자잘한 포장재,진쪼 좋아합니다 맛있고,다음 토욜
8,포장 꼼꼼하고 유통기간,제품 건강한 단맛,개는 깨져서
9,맛있어요 요거트 만들어져요,빠르고 자잘한 포장재,폭풍 성장


In [14]:
df['KeyBERT'] =0
for t in keywords3:
    temp = list(t.split())
    text1, text2, text3 = temp[0],  temp[1],  temp[-1]
    try:
        df.loc[(df['REVIEW'].str.contains(str(text1))) & (df['REVIEW'].str.contains(str(text2))) & (df['REVIEW'].str.contains(str(text3))) , 'KeyBERT'] = 1
    except:
        df.loc[(df['리뷰상세내용'].str.contains(str(text1))) & (df['리뷰상세내용'].str.contains(str(text2))) & (df['리뷰상세내용'].str.contains(str(text3))) , 'KeyBERT'] = 1

In [18]:
df[df['KeyBERT'] == 1].tail(2)

,Unnamed: 0,GROUP2,KEYWORD,리뷰상세내용,구매자평점,포토/영상,등록자,리뷰등록일1,상품번호,상품명,product_nm,token,token_attribution2,lenght,KeyBERT
353,38440,제품포장,포장,포장 잘 되있어요\n간식용으로 구매해서 불필요한 플라스틱케이스 없어서 좋아요,5,https://phinf.pstatic.net/checkout.phinf/20230...,ssk9***,2023-03-23,4697498993,페레로 로쉐 T3 3개입 16세트 (총 48개입) (쇼핑백 없는 구성),페레로 [ 로쉐 ] [ t3 ],포장 자다 되다 어요 간 식용 으로 구매 해서 불 필요 한 플라스틱 케이스 없다 어...,포장/Noun 자다/Verb 되다/Verb 어요/Noun 간/Noun 식용/Noun...,17,1
422,48389,제품포장,녹다,물에잘녹고 아기가잘먹어요\n덕분에 폭풍성장중입니다~^^,5,NaN,rkgb***,2023-03-24,6510886109,앱솔루트 센서티브 2단계(100일~6개월) 900g 3캔,분유 앱솔루트 [ 센서티브 ] [ 2단계 ] [ 900g ],물 에 자다 녹다 아기 가 잘 먹다 어요 덕분 폭풍성 장 중 이다,물/Noun 에/Josa 자다/Verb 녹다/Verb 아기/Noun 가/Josa 잘...,15,1
